In [3]:
from torchvision import models

![](./img/imagenet.png)

# 1. Alexnet

Мы уже упоминали нейросеть AlexNet в прошлой лекции, когда рассказывали о соревновании ImageNet, где она достигла прорывных для своего времени результатов. Её архитектура состоит из пяти свёрточных слоёв, между которыми располагаются pooling-слои и слои нормализации, а завершают нейросеть три полносвязных слоя.

![](./img/alexnet.png)

In [4]:
net = models.alexnet(pretrained=False)

![](./img/stack_layers.png)

# 2. VGG

На диаграмме можно видеть, что в 2014 году VGGNet достигла более чем в два раза лучшего результата по сравнению с AlexNet. Основная идея VGG-архитектур — использование большего числа слоёв с фильтрами меньшего размера. Существуют версии VGG-16 и VGG-19 с 16 и 19 слоями соответственно.

![](./img/vgg.png)

In [5]:
net = models.vgg16(pretrained=False)
net = models.vgg16_bn(pretrained=False) # добавили batchnorm

net = models.vgg19(pretrained=False)
net = models.vgg19_bn(pretrained=False) # добавили batchnorm

# 3. Inception (GoogleNet)

## 3.1 v1

Пререквезит:
- Интересуемые классы на изображении могут иметь крайне большую разницу в размерах, например собаки на следующих изображениях

![](./img/dog_sizes.jpeg)

- Из-за такого огромного разнообразия в расположении информации выбор правильного размера ядра для операции свертки становится затруднительным. Ядро большего размера предпочтительно для классов, которые имеют большой размер, а ядро меньшего размера предпочтительно для классов, которая имеют меньший размер.

- Глубокие нейронные сети склонны к переобучению и градиент течет в них гораздо хуже

$$\large{(\frac{\partial F_1}{\partial W_1} < 1)(\frac{\partial F_2}{\partial W_2} < 1) \dots (\frac{\partial F_N}{\partial W_N} < 1)}$$

- Простое добавление сверточных слоев ведет к большому потреблению вычислительных ресурсов


Решение:
Давайте сделаем несколько сверток которые работают на одном уровне с разными ядрами

![](./img/inception1.png)

## 3.2 v2

Можно ли как то еще уменьшить вычислительную сложность, давайте обратим внимание на то что свертка 5х5 имеет такую же карту признаков как и 2 последовательные свертки 3х3, только вместо 25 параметров в ядре, у нас будет 2 по 9 = 18

![](./img/inception2_pre.png)

А теперь заметим что свертку 3х3 можно разложить как две свертки 1х3 и 3х1 и получить 6 параметров вместо 18

![](./img/inception2.png)

![](./img/googlenet.png)

## 3.3 v3

- Теже блоки что у 2й версии
- BatchNorm + Dropout в классификаторе
- LabelSmoothing в лосс функции

$$\large{y_{smooth} = (1 - \alpha) y_{one\_hot} + \frac{\alpha}{C}}$$

- C - число классов
- $y_{one\_hot}$ - наш one hot вектор ответа
- $\alpha$ - параметр размытия от 0 до 1, в 0 - стандартная cross entropy в 1 все классы равновероятны

In [7]:
net = models.inception_v3(pretrained=False)

/home/imd/miniconda3/envs/mlisuct/lib/python3.8/site-packages/torchvision/models/inception.py:81: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of inception_v3 will be changed in future releases of '


# 4 DenseNet

DenseNet это очередная архитектура сверточной сети, она использует dense связи между слоями, через имплементацию Dense Blocks, в котором мы соеденяем все слои (с сохранением размеров крат признаков) напрямую друг с другом. Чтобы сохранить природу прямого прохождения, каждый слой получает дополнительные входные данные от всех предыдущих слоев и передает свои собственные карты признаков всем последующим слоям. Мотивация схожа с inception - мы пытаемся сохранить информацию о более маленьких объектах и облегчить течение градиента


## 4.1 DenseBlock 

Это блок в котором реализована dense связь между слоями, каждый вход предыдущего слоя конкатенируется с входом текущего слоя в блоке, если у нас имеется L блоков, тогда количество соедениний в нем будет равно:

$$\large{\frac{L(L+1)}{2}}$$

![](./img/dense_block.png)

![](./img/densenet.ppm)

In [8]:
net = models.densenet121(pretrained=False)
net = models.densenet201(pretrained=False)

# 5 ResNet

В декабре 2015-го, примерно в то же время, как была представлена архитектура Inception v3, произошла революция — опубликовали ResNet. В ней заложены простые идеи: подаём выходные данные двух успешных свёрточных слоёв И обходим входные данные для следующего слоя

![](./img/resnet_block.jpg)

Такие идеи уже предлагались, например, <a href="http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf">здесь</a>. Но в данном случае авторы обходят ДВА слоя и применяют подход в больших масштабах. Обход одного слоя не даёт особой выгоды, а обход двух — ключевая находка. Это можно рассматривать как маленький классификатор, как сеть-в-сети!

Также это был первый в истории пример обучения сети из нескольких сотен, даже тысячи слоёв.
В многослойной ResNet применили bottleneck-слой, аналогичный тому, что применяется в Inception:

![](./img/bottleneck.jpg)

Этот слой уменьшает количество свойств в каждом слое, сначала используя свёртку 1х1 с меньшим выходом (обычно четверть от входа), затем идёт слой 3х3, а потом опять свёртка 1х1 в большее количество свойств. Как и в случае с Inception-модулями, это позволяет экономить вычислительные ресурсы, сохраняя богатство комбинаций свойств. Сравните с более сложными и менее очевидными stem-ами в Inception V3

В небольших resnet до resnet52 не используется хак с уменьшением признаков в блоке

![](./img/resnet18.png)

В больших resnet начиная с resnet101 используется хак

![](./img/resnet101.png)

In [9]:
net = models.resnet50(pretrained=False)
net = models.resnet101(pretrained=False)

# 6. Se-ResNet

Не все признаки одинаково важны, поэтому давайте будем взвешивать наши признаки после ResNet блока с помощью Sigmoid (применяем global pooling - берем максимум в каждом канале от всей карты признаков). И далее выход Resnet мы взвесим по нашим выходам из Sigmoid

![](./img/se_resnet.png)

In [10]:
# данной модели нет в torchvision но можно заменить bootleneck самому

# 7 ResNext

ResNeXt блок состоит из повторяющихся частей которые объединяют набор преобразований с одной и той же топологией. В сравнении ResNet, у нас появляется новое измерение - кардинальность (размер набора преобразований) C в качестве важного фактора в дополнение к измерениям глубины и ширины.

![](./img/resnext.png)

$$\large{resnet\_features = 256 \cdot 1 \cdot 1 \cdot 64 + 64 \cdot 3 \cdot 3 \cdot 64 + 64 \cdot 1 \cdot 1 \cdot 256 = 69632}$$
$$\large{resnext\_features = 256 \cdot 1 \cdot 1 \cdot 4 \cdot 32 + 4 \cdot 3 \cdot 3 \cdot 4 \cdot 32 + 4 \cdot 1 \cdot 1 \cdot 256 \cdot 32 = 70144}$$

In [14]:
net = models.resnext50_32x4d(pretrained=False, num_classes=2)

# 8. Se-ResNext

Объеденяем идеи resnext и se-resnet

![](./img/se_resnext.png)

In [13]:
# модель отсутсвует по причине отсутствия se блока